In [97]:

import pandas as pd
from dotenv import load_dotenv
load_dotenv()
import json
import numpy as np




In [98]:
# DATASET = "zero_shot_cot_50_50_merged"
# DATASET = "baseline_zero_shot_gpt3_50_50_merged"
DATASET = "auto_gpt3_gpt3_50_50_generations_merged"


GENERATIONS_PATH = "../datasets/error_analysis/" + DATASET + ".csv"
OUTPUT_PATH = "../datasets/error_analysis/structured_explanation_analysed/" + DATASET + "_no_errors.csv"



data = pd.read_csv(GENERATIONS_PATH)
data.head()

,Unnamed: 0,traceId,model,startTime,endTime,prompt,completion,eval,predicted_label,label,error,similarity,confidence,structured_explanation_v2,structured_explanation_v2_gpt4
0,0,136e98ba-ac14-42f4-8345-103a04fd95e3,gpt-3.5-turbo-0301,2023-10-20T12:39:55.032Z,2023-10-20T12:40:03.460Z,Q: Are the following two products the same?\r\...,"First, we can compare the titles and descripti...",no match,False,True,FN,20%,90%,"Based on the information provided, the attribu...","Based on the information provided, the attribu..."
1,1,136e98ba-ac14-42f4-8345-103a04fd95e3,gpt-3.5-turbo-0301,2023-10-20T12:39:42.734Z,2023-10-20T12:39:53.607Z,Q: Are the following two products the same?\r\...,"First, we can compare the titles and descripti...",The two products may match.,True,True,NONE,90%,80%,"[{""attribute"":""brand"",""importance"":""0.10"",""val...","[{""attribute"":""brand"",""importance"":""1.00"",""val..."
2,2,136e98ba-ac14-42f4-8345-103a04fd95e3,gpt-3.5-turbo-0301,2023-10-20T12:39:28.325Z,2023-10-20T12:39:40.801Z,Q: Are the following two products the same?\r\...,"First, we can compare the titles and descripti...",Match.,True,True,NONE,90%,80%,NaN,"Based on the information provided, the attribu..."
3,3,136e98ba-ac14-42f4-8345-103a04fd95e3,gpt-3.5-turbo-0301,2023-10-20T12:39:10.835Z,2023-10-20T12:39:26.196Z,Q: Are the following two products the same?\r\...,"First, we can compare the titles and descripti...",Match.,True,True,NONE,60%,80%,NaN,"Based on the information provided, here is a s..."
4,4,136e98ba-ac14-42f4-8345-103a04fd95e3,gpt-3.5-turbo-0301,2023-10-20T12:38:55.776Z,2023-10-20T12:39:08.718Z,Q: Are the following two products the same?\r\...,"First, we can compare the titles of the two pr...",The two products may match.,True,True,NONE,90%,80%,NaN,"[{""attribute"":""brand"",""importance"":""0.5"",""valu..."


In [99]:
# remove the rows with no error
data = data[data["error"] == "NONE"]

In [100]:
list = []

counter = 1
for i, row in enumerate(data["structured_explanation_v2_gpt4"]):
    try:
        # cut of the text and extract the json
        preprocessed_row = row[row.find("["): row.rfind("]")+1]
        row_json = json.loads(preprocessed_row)
        list.append(row_json)
    except Exception as e1:
        try:
            print(counter)
            counter += 1
            fixed = row.replace(",\n"+ row.split("\n")[-1], "]")
            row_json = json.loads(fixed)
            list.append(row_json)
            # print("fixed:" + str(i))
        except Exception as e2:
            print("-----")
            print(e2)
            print("Here: \n\n")
            print(preprocessed_row)
            # print(i)
            print("-----")


In [101]:
len(list)

208

In [102]:

# Create empty dictionaries to store statistics
attribute_counts = {}
attribute_total_importance = {}
attribute_importances = {}
count = 0
# Iterate through the list of JSONs
for item in list:
    for item2 in item:
        count += 1
        attribute = item2['attribute']
        importance = float(item2['importance'])
        # values = item2['values']

        # Update the dictionaries
        if attribute in attribute_counts:
            attribute_counts[attribute] += 1
            attribute_total_importance[attribute] += importance
            attribute_importances[attribute].append(importance)
        else:
            attribute_counts[attribute] = 1
            attribute_total_importance[attribute] = importance
            attribute_importances[attribute] = [importance]
print(count)
# Calculate average and variance
attribute_average_importance = {attribute: total_importance / attribute_counts[attribute] for attribute, total_importance in attribute_total_importance.items()}
attribute_variance_importance = {attribute: np.var(importances) for attribute, importances in attribute_importances.items()}



# Create a DataFrame from the calculated statistics
attribute_stats = pd.DataFrame({
    'attribute': attribute_counts.keys(),
    'count': attribute_counts.values(),
    'average importance': [attribute_average_importance[attr] for attr in attribute_counts.keys()],
    'variance of importance': [attribute_variance_importance[attr] for attr in attribute_counts.keys()]
})
# sum of counts
print(attribute_stats['count'].sum())
# Display the resulting table
attribute_stats



1088
1088


,attribute,count,average importance,variance of importance
0,brand,197,0.411168,0.084418
1,model,141,0.544681,0.117826
2,size,20,0.425000,0.082875
3,casing,5,0.520000,0.060600
4,compound,3,0.483333,0.093889
...,...,...,...,...
220,finish,1,0.250000,0.000000
221,grain quality,1,0.250000,0.000000
222,image formation,1,0.250000,0.000000
223,resistance to age-induced changes,1,0.250000,0.000000


In [103]:
# sort by count
attribute_stats = attribute_stats.sort_values(by=['count'], ascending=False)
attribute_stats.head(30)

,attribute,count,average importance,variance of importance
0,brand,197,0.411168,0.084418
1,model,141,0.544681,0.117826
36,product type,81,0.704938,0.056827
17,color,50,0.060000,0.021700
7,type,45,0.490000,0.115400
20,capacity,45,0.257778,0.052106
14,interface,29,0.217241,0.048496
15,speed,23,0.526087,0.086276
2,size,20,0.425000,0.082875
19,compatibility,18,0.388889,0.074043


In [104]:
attribute_stats.to_csv(OUTPUT_PATH, index=False)